In [14]:
import requests 
import json
import pprint as pp # pretty print

import aries_cloudagent.wallet.crypto as crypto
import base64
import base58
import uuid
import hashlib
import time
import jupyter_util as util
from importlib import reload
reload(util)

<module 'jupyter_util' from '/home/sevni/Documents/thclab/aries/controller_acapy_and_tests/jupyter_util.py'>

In [15]:
localhost = "http://localhost:"

def assert_status_code_is_200(response_object):
    get = response_object
    if get.status_code != 200:
        print("status_code: " + str(get.status_code) + "  value: " + str(get.text))
        assert get.status_code == 200, "status_code is not 200"

def get(port, url_end) -> dict:
    port = str(port)
    request = requests.get(localhost + port + url_end)
    assert_status_code_is_200(request)
    result = json.loads(request.text)
    return result
    
def post(port, url_end, json_dict=None) -> dict:
    port = str(port)
    if json_dict != None:
        request = requests.post(localhost + port + url_end, json=json_dict)
    else:
        request = requests.post(localhost + port + url_end)
        
    assert_status_code_is_200(request)
    result = json.loads(request.text)
    return result    

In [27]:
def save_to_pds(some_json):
    add = requests.post('http://localhost:8150/pds/save', json=some_json)
    assert_status_code_is_200(add)
    
    add = json.loads(add.text)
    return add

def save_to_pds2(some_json):
    add = requests.post('http://localhost:8151/pds/save', json=some_json)
    assert_status_code_is_200(add)
    
    add = json.loads(add.text)
    return add

def get_from_pds(some_id):
    get = requests.get('http://localhost:8150/pds/' + some_id)
    assert_status_code_is_200(get)    
        
    get = json.loads(get.text)
    return get

def get_from_pds2(some_id):
    get = requests.get('http://localhost:8150/pds/' + some_id)
    assert_status_code_is_200(get)    
        
    get = json.loads(get.text)
    return get

def save_and_retrieve(optional_test_string = ""):
    payload = {"payload": "testing" + " " + optional_test_string}
    id = save_to_pds(payload)
    content = get_from_pds(str(id['payload_id']))
    
    assert payload == content, f"initial payload not equal to return value from pds {payload, content}"
    
    return id['payload_id']

def get_pdses():
    get1 = requests.get('http://localhost:8150/pds')
    get2 = requests.get('http://localhost:8151/pds')
    
    assert get1.text != None and get2.text != None
    assert_status_code_is_200(get1)
    assert_status_code_is_200(get2)
    
    return [get1.text, get2.text]
        
        
def set_pds_config(pds_type, settings = {}, instance_name = "default"):
    if settings != {}:
        settings.update({"optional_instance_name": instance_name})
        config = requests.post('http://localhost:8150/pds/settings', json={
          "settings": {
              pds_type: settings
          }
        })
        assert_status_code_is_200(config)    
        
    config = requests.post('http://localhost:8150/pds/activate?type=' + pds_type + '&optional_name=' + instance_name)
    assert_status_code_is_200(config)   
    
    if settings != {}:
        settings.update({"optional_instance_name": instance_name})
        config = requests.post('http://localhost:8151/pds/settings', json={
          "settings": {
              pds_type: settings
          }
        })
        assert_status_code_is_200(config)    
        
    config = requests.post('http://localhost:8151/pds/activate?type=' + pds_type + '&optional_name=' + instance_name)
    assert_status_code_is_200(config) 
    
    
def set_own_your_data():
    set_pds_config("own_your_data", {
        "client_id": "8SX1-RBazhic9ftG4HyG2XegrQ2kdLGu0hd-Ty3IZnE", 
        "client_secret": "gEg9_tTh2V1ZK6piL9tZ38YO-1xqXLDXCccJS4uQUJg",
        "api_url": "https://data-vault.eu"
    })
    
def activate_pds_both_agents(pds_type, instance_name = 'default'):
    config = requests.post('http://localhost:8150/pds/activate?type=' + pds_type + '&optional_name=' + instance_name)
    assert_status_code_is_200(config)   
    config = requests.post('http://localhost:8151/pds/activate?type=' + pds_type + '&optional_name=' + instance_name)
    assert_status_code_is_200(config) 
    
def test_pds_return_same_id():
    set_own_your_data()
    id1 = save_to_pds({"payload": "testing"})
    set_pds_config("local")
    id2 = save_to_pds({"payload": "testing"})
    
    assert id1['payload_id'] == id2['payload_id'], "own_your_data and local return different ids"
    
    set_pds_config("local")
    id1 = save_to_pds({"payload": "testing"})
    set_pds_config("data_vault", {"host": "http://ocadatavault"})
    id2 = save_to_pds({"payload": "testing"})
    
    assert id1['payload_id'] == id2['payload_id'], "local and data_vault return different ids"

    return [id1, id2]
    
    
# test_pds_return_same_id()
set_pds_config("local")
save_and_retrieve("local")
set_pds_config("data_vault", {"api_url": "http://ocadatavault"})
save_and_retrieve("data_vault_thcf")
set_pds_config("data_vault", {"api_url": "http://ocadatavault"}, "vault2")
save_and_retrieve("data_vault_thcf_2")
set_own_your_data()
payload_id = save_and_retrieve("own_your_data")


print("PDS agent1 and agent2", get_pdses())



PDS agent1 and agent2 ['{"active": "own_your_data, default", "types": {"local": {"oca_schema_namespace": "pds", "oca_schema_dri": "3Fb68s1EPcX4HZhhT23HXrYpuMfcZdreD8xNmEMDc6nC"}, "data_vault": {"oca_schema_namespace": "pds", "oca_schema_dri": "ejHFuhg2v1ZrL5uQrHe3Arcxy62GWNakjTwL38swC9RB"}, "own_your_data": {"oca_schema_namespace": "pds", "oca_schema_dri": "9bABtmHu628Ss4oHmyTU5gy7QB1VftngewTmh7wdmN1j"}}}', '{"active": "own_your_data, default", "types": {"local": {"oca_schema_namespace": "pds", "oca_schema_dri": "3Fb68s1EPcX4HZhhT23HXrYpuMfcZdreD8xNmEMDc6nC"}, "data_vault": {"oca_schema_namespace": "pds", "oca_schema_dri": "ejHFuhg2v1ZrL5uQrHe3Arcxy62GWNakjTwL38swC9RB"}, "own_your_data": {"oca_schema_namespace": "pds", "oca_schema_dri": "9bABtmHu628Ss4oHmyTU5gy7QB1VftngewTmh7wdmN1j"}}}']


In [30]:
## TEST that Settings are persistent! 
activate_pds_both_agents("data_vault")
save_and_retrieve("data_vault_thcf")
activate_pds_both_agents("data_vault", "vault2")
save_and_retrieve("data_vault_thcf_2")
activate_pds_both_agents("own_your_data")
payload_id = save_and_retrieve("own_your_data")

In [31]:
agent1 = {}
plugin = {}
endpoint = "http://localhost:8150/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent1['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent1['url'])

agent2 = {}
endpoint = "http://localhost:8151/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent2['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent2['url'])

agent1 = util.processInviteUrl(agent1['url'])
agent2 = util.processInviteUrl(agent2['url'])
print("Decoded invitation:\n")
pp.pprint(agent2['invite'])

### Generating a did key using aries crypto module
# Create a key pair with random seed 
# Order: 0 - public key, 1 - secret / pritvate key
plugin['keypair'] = crypto.create_keypair()
# it seems to me that did is an encoded sub string of public key
# or maybe just some arbitrary random number
did = plugin['keypair'][0][:16]
# final version of our did
plugin['did'] = base58.b58encode(did).decode("ascii")
# encoding keys to base58
plugin['public_key_b58'] = \
    base58.b58encode(plugin['keypair'][0]).decode("ascii")
plugin['private_key_b58'] = \
    base58.b58encode(plugin['keypair'][1]).decode("ascii")

print("Private key: \n", plugin['private_key_b58'])
print("\nPublic key: \n", plugin['public_key_b58'])

agent1['connection'] = util.connectWithAcapy(agent1, plugin)
agent2['connection'] = util.connectWithAcapy(agent2, plugin)

agent1['connection_details'] = util.decodeConnectionDetails(agent1['connection'])
agent2['connection_details'] = util.decodeConnectionDetails(agent2['connection'])
agents = [
    {
        "label": agent1['invite']['label'],
        "DIDDoc": agent1['connection_details']['DIDDoc'],
        "myKey": plugin['keypair'][1]
    },
    {
        "label": agent2['invite']['label'],
        "DIDDoc": agent2['connection_details']['DIDDoc'],
        "myKey": plugin['keypair'][1]
    },
]

connection = util.connectAgents(agents[0], agents[1])
connection[1] = connection[1]['raw_repr']['connection_id']
connection[0] = connection[0]['connection_id']
print(connection[0], "\n", connection[1])

http://agent1.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiNjk3Njc2ZmQtYTAzYi00ZTYwLWI0ZmItYzUzMWJjZDhlMDM4IiwgImxhYmVsIjogIk1haW4gKGFkbWluKSIsICJzZXJ2aWNlRW5kcG9pbnQiOiAiaHR0cDovL2FnZW50MS5sb2NhbGhvc3QiLCAicmVjaXBpZW50S2V5cyI6IFsiM25kYnRiOFZOTmVlYUtNMUZVQjRDNFEzZXNFSm5hNDg0azJmYnJpblpmWmUiXX0=
http://agent2.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiY2ZlNjM0MjMtOTZjNS00YmJiLWEwZGEtNTMzNmM3ZjVmMjdjIiwgInNlcnZpY2VFbmRwb2ludCI6ICJodHRwOi8vYWdlbnQyLmxvY2FsaG9zdCIsICJyZWNpcGllbnRLZXlzIjogWyI2eVBRdVdYVlBLRlNHZVJFcmJZUnpieGJNd0tCbXppdWN0ZEd3UFNCZEdpaCJdLCAibGFiZWwiOiAiQ2xpZW50IChhZG1pbikifQ==
Decoded invitation:

{'@id': 'cfe63423-96c5-4bbb-a0da-5336c7f5f27c',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
 'label': 'Client (admin)',
 'recipientKeys': ['6yPQuWXVPKFSGeRErbYRzbxbMwKBmziuctdGwPSBdGih'],
 

In [6]:
# time.sleep(0.2)

# add = requests.post(f'http://localhost:8151/pds/get_from?connection_id={connection[1]}&payload_id={1234}')
# print(add.text)

# set_pds_config("own_your_data", {
#     "client_id": "8SX1-RBazhic9ftG4HyG2XegrQ2kdLGu0hd-Ty3IZnE", 
#     "client_secret": "gEg9_tTh2V1ZK6piL9tZ38YO-1xqXLDXCccJS4uQUJg"
# })
# payload_id = save_and_retrieve("own_your_data1234")
# add = requests.post(f'http://localhost:8151/pds/get_from?connection_id={connection[1]}&payload_id={payload_id}')
# assert '{"message_sent": "success"}' == add.text

# time.sleep(1)
# get1 = requests.get('http://localhost:8150/pds/' + str(payload_id))
# get2 = requests.get('http://localhost:8151/pds/' + str(payload_id))
# print(get1.text, get2.text)
# assert get1.status_code == get2.status_code, f"status codes are not equal {get1.status_code, get2.status_code}"
# assert get1.text == get2.text, "exchanged data with get_from is not equal"
# set_pds_config("data_vault", {"host": "http://ocadatavault"})

### FEATURE DISCOVERY

In [117]:
time.sleep(0.5)
newDid1 = util.createDid(agents[1])
newDid0 = util.createDid(agents[0])

try:
    util.registerDidOnLedger(agents[1], newDid1['result']['did'], newDid1['result']['verkey'])
except:
    pass
try:
    util.registerDidOnLedger(agents[0], newDid0['result']['did'], newDid0['result']['verkey'])
except:
    pass

util.setPublicDid(agents[1], newDid1['result']['did'])
util.setPublicDid(agents[0], newDid0['result']['did'])
time.sleep(0.5)

In [99]:
issue_cred = post(8150, '/issue-credential/request', {
  "credential_values": {
    "type": "Person",
    "address": {
      "type": "PostalAddress",
      "streetAddress": "123 Main St.",
      "addressLocality": "Blacksburg",
      "addressRegion": "VA",
      "postalCode": "24060",
      "addressCountry": "US",
    }
  },
  "connection_id": connection[0],
})

pp.pprint(issue_cred)

{'credential_exchange_id': '48f96ad2-6ba8-42a5-a94f-2c17dbe223b2'}


In [100]:
cred_exchange = get(8151, "/issue-credential/exchange/record?state=request_received")
print(cred_exchange)

[{'initiator': 'external', 'connection_id': '4e10d704-658f-42e1-81ce-3d7f8b483581', 'updated_at': '2020-11-30 09:13:49.741527Z', 'trace': False, 'role': 'issuer', 'state': 'request_received', 'credential_exchange_id': 'a6a85fa5-c3b9-4d3d-b17f-78e2b544ce6c', 'created_at': '2020-11-30 09:13:49.741527Z', 'thread_id': '404d6a65-f536-436d-863a-2b317d58a9c6', 'credential_request': {'credential_values': {'type': 'Person', 'address': {'type': 'PostalAddress', 'streetAddress': '123 Main St.', 'addressLocality': 'Blacksburg', 'addressRegion': 'VA', 'postalCode': '24060', 'addressCountry': 'US'}}}}]


In [101]:
id = cred_exchange[0]['credential_exchange_id']
issue_cred = post(8151, "/issue-credential/issue?credential_exchange_id=" + id)

pp.pprint(issue_cred)

('{"context": ["https://www.w3.org/2018/credentials/v1", '
 '"https://www.schema.org"], "type": ["VerifiableCredential"], "issuer": '
 '"6H9XUXosoX8X14Uuxj3tek", "issuanceDate": "2020-11-30 09:13:50.930087Z", '
 '"credentialSubject": {"type": "Person", "address": {"type": "PostalAddress", '
 '"streetAddress": "123 Main St.", "addressLocality": "Blacksburg", '
 '"addressRegion": "VA", "postalCode": "24060", "addressCountry": "US"}, "id": '
 '"WzJvwbcn8BU9Br6cr7o8yu"}, "proof": {"jws": '
 '"ZqjaUPy6_Zmj7J-nZJ_u3JfvJsFytzq_pTd5gpy3mQWumWmHJe_X5y-6cKJOZ_oydHtEFHPBrkMuGnf0gBqWDA", '
 '"type": "Ed25519Signature2018", "created": "2020-11-30 09:13:50.949729Z", '
 '"proofPurpose": "assertionMethod", "verificationMethod": '
 '"B7JNvK1HXiSkgkUBm2pD4e3ayxZ9xjxE7xioWGh5d8PS"}}')


In [102]:
issue_cred = post(8151, '/present-proof/request', {
  "connection_id": connection[1],
  "schema_base_dri": "12345",
  "requested_attributes": ["address"],
  "issuer_did": "123145"
})

pp.pprint(issue_cred)

{'thread_id': 'b63afc24-51e9-4eb4-be6c-7b0b293c0528'}


In [103]:
exchange_record = get(8150, f"/present-proof/exchange/record?state=request_received&connection_id={connection[0]}&thread_id={issue_cred['thread_id']}")

In [104]:
creds = get(8150, "/credentials")
issue_cred = post(8150, '/present-proof/present', {
  "credential_id": creds['results'][0]['id'],
  "exchange_record_id": exchange_record[0]['presentation_exchange_id']
})

pp.pprint(issue_cred)

'success, proof sent and exchange updated'


# Services

In [126]:
data_dri = save_to_pds2({"payload": "testing"})
add = post(8151, '/verifiable-services/add', {
  "label": "consent_test",
  "service_schema": {
    "oca_schema_namespace": "string",
    "oca_schema_dri": "string"
  },
  "consent_schema": {
    "oca_schema_namespace": "consent",
    "oca_schema_dri": "fArVHJTQSKHu2CeXJocQmH3HHxzZXsuQD7kzyHJhQ49s",
    "data_dri": data_dri['payload_id'],
  }
})
print(add)

{'service_schema': {'oca_schema_dri': 'string', 'oca_schema_namespace': 'string'}, 'created_at': '2020-11-30 09:46:30.833874Z', 'consent_schema': {'oca_schema_dri': 'fArVHJTQSKHu2CeXJocQmH3HHxzZXsuQD7kzyHJhQ49s', 'oca_schema_namespace': 'consent', 'data_dri': 'zQmcJf1w9bVpquGdzCp86pX4K21Zcn7bJBUtrBP1cr2NFuR'}, 'label': 'consent_test', 'updated_at': '2020-11-30 09:46:30.833874Z'}


In [127]:
service_list = requests.get('http://localhost:8150/verifiable-services/DEBUGrequest/' + connection[0])
request_service_list = requests.get('http://localhost:8150/verifiable-services/request-service-list/' + connection[0])
print(request_service_list.text)

print(service_list)
service_list = json.loads(service_list.text)
pp.pprint(service_list)

"SUCCESS: request sent, expect a webhook notification"
<Response [200]>
{'connection_id': '47ecfa5c-4109-431f-9a81-f7b7879069b3',
 'created_at': '2020-11-30 09:41:02.180021Z',
 'services': [{'consent_schema': {'data_dri': 'zQmcJf1w9bVpquGdzCp86pX4K21Zcn7bJBUtrBP1cr2NFuR',
                                  'oca_schema_dri': 'fArVHJTQSKHu2CeXJocQmH3HHxzZXsuQD7kzyHJhQ49s',
                                  'oca_schema_namespace': 'consent'},
               'label': 'consent_test',
               'service_id': '2995e49d-275c-452d-80e0-c1b699b0e5bb',
               'service_schema': {'oca_schema_dri': 'string',
                                  'oca_schema_namespace': 'string'}},
              {'consent_schema': {'data_dri': 'zQmcJf1w9bVpquGdzCp86pX4K21Zcn7bJBUtrBP1cr2NFuR',
                                  'oca_schema_dri': 'fArVHJTQSKHu2CeXJocQmH3HHxzZXsuQD7kzyHJhQ49s',
                                  'oca_schema_namespace': 'consent'},
               'label': 'consent_test',
         

In [128]:
apply = requests.post('http://localhost:8150/verifiable-services/apply', json={
  "connection_id": connection[0],
  "service_id": service_list['services'][0]['service_id'],
  "user_data": "abckonoin",
  "service": service_list['services'][0],
})
apply = json.loads(apply.text)
print(apply)

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/verifiable-services/1.0/application', '@id': 'e70cb5f9-138c-4740-91ae-e8955f25a85e', 'exchange_id': '20d4bdd7-62ef-426f-8b39-324614157412', 'service_user_data': 'abckonoin', 'service_user_data_dri': 'zQmSnRDrp3sNzsB194RaKwqKWmFS7mbT8oiF7qMWUCoNGgQ', 'service_id': '2995e49d-275c-452d-80e0-c1b699b0e5bb', 'consent_credential': '{"context": ["https://www.w3.org/2018/credentials/v1", "https://www.schema.org"], "type": ["VerifiableCredential"], "issuer": "Wc6p1JmuMp7MTDudULpCx9", "issuanceDate": "2020-11-30 09:46:32.731359Z", "credentialSubject": {"oca_schema_dri": "fArVHJTQSKHu2CeXJocQmH3HHxzZXsuQD7kzyHJhQ49s", "oca_schema_namespace": "consent", "data_dri": "zQmcJf1w9bVpquGdzCp86pX4K21Zcn7bJBUtrBP1cr2NFuR", "service_consent_match_id": "86144209-f8b8-4afe-a5f3-b351bdef7bc2", "id": "HDLnczx2HHX5qUBntRyDHS"}, "proof": {"jws": "Iq5dQEI_-MWM6Z5rSiNNjKPEycAyzsnYxFRLuKxKZb93pdxxDnBJ_hsM8KRVnz8zo7jlH5sWFTYASXqSz_ZvCA", "type": "Ed25519Signature2018", "

In [129]:
time.sleep(1)
issue_self = requests.post('http://localhost:8151/verifiable-services/get-issue', json={
    "exchange_id": apply['exchange_id'],
})
print(issue_self)
issue_self = json.loads(issue_self.text)
pp.pprint(issue_self)

<Response [200]>
[{'author': 'other',
  'connection_id': '90b2abec-c3f9-4da6-acb9-9f18dbae5e6f',
  'consent_credential': "{'context': "
                        "['https://www.w3.org/2018/credentials/v1', "
                        "'https://www.schema.org'], 'type': "
                        "['VerifiableCredential'], 'issuer': "
                        "'Wc6p1JmuMp7MTDudULpCx9', 'issuanceDate': '2020-11-30 "
                        "09:46:32.731359Z', 'credentialSubject': "
                        "{'oca_schema_dri': "
                        "'fArVHJTQSKHu2CeXJocQmH3HHxzZXsuQD7kzyHJhQ49s', "
                        "'oca_schema_namespace': 'consent', 'data_dri': "
                        "'zQmcJf1w9bVpquGdzCp86pX4K21Zcn7bJBUtrBP1cr2NFuR', "
                        "'service_consent_match_id': "
                        "'86144209-f8b8-4afe-a5f3-b351bdef7bc2', 'id': "
                        "'HDLnczx2HHX5qUBntRyDHS'}, 'proof': {'jws': "
                        "'Iq5dQEI_-MWM6Z5rSiNNjKP

In [130]:
issue_self = requests.post('http://localhost:8151/verifiable-services/process-application', json={
    "issue_id": issue_self[0]['issue_id'],
    "decision": "accept",
})
print(issue_self)
issue_self = json.loads(issue_self.text)
pp.pprint(issue_self)

<Response [200]>
{'connection_id': '90b2abec-c3f9-4da6-acb9-9f18dbae5e6f',
 'issue_id': 'ba04951647b85860ea89c7ddc6d439cea5891c60e15035a170279edc1a92909c'}


In [131]:
credentials = requests.get('http://localhost:8150/credentials')
credentials = json.loads(credentials.text) 
pp.pprint(credentials)

{'results': [{'credential': {'context': ['https://www.w3.org/2018/credentials/v1',
                                         'https://www.schema.org'],
                             'credentialSubject': {'data_dri': 'zQmSnRDrp3sNzsB194RaKwqKWmFS7mbT8oiF7qMWUCoNGgQ',
                                                   'id': 'Wc6p1JmuMp7MTDudULpCx9',
                                                   'oca_schema_dri': 'string',
                                                   'oca_schema_namespace': 'string',
                                                   'service_consent_match_id': '82b98928-806e-47fa-8142-0a9fb98cf110'},
                             'issuanceDate': '2020-11-30 09:44:12.871269Z',
                             'issuer': 'HDLnczx2HHX5qUBntRyDHS',
                             'proof': {'created': '2020-11-30 09:44:12.890078Z',
                                       'jws': 'PJiMlHB8oTXtf3uZjlA6JN2ZjyB2-FXqtWOwbctBkbo21RTB9lOxVl8NpegbK1LUn9Wlhfthg347tEXNGFT1Ag',
          

In [141]:
proof_schema = {
  "proof_request": {
    "version": "1.0",
    "name": "Proof request",
    "requested_attributes": {
      "data_dri": {
        "non_revoked": {
          "from_epoch": 1500000000,
          "to_epoch": 1700758817
        },
        "restrictions": [
          {
            "cred_def_id": apply['credential_definition_id'],
          }
        ],
        "name": "data_dri"
      },
    },
    "requested_predicates": {
    }
  },
  "connection_id": connection[1],
#   "trace": True,
  "comment": "string",
  "nonce": "1234567890"
}
proof_json = json.dumps(proof_schema)

proof = requests.post('http://localhost:8151/present-proof/send-request', json=proof_schema)
print(proof.text)

KeyError: 'credential_definition_id'

In [359]:
proof = requests.get('http://localhost:8150/present-proof/records')
proof_requests = json.loads(proof.text)['results']
proof_request = proof_requests[-4]
pp.pprint(proof_request)

{'connection_id': '994ebd0f-14c2-428c-be53-e730bc3ca7a7',
 'created_at': '2020-09-22 11:58:22.954779Z',
 'initiator': 'external',
 'presentation_exchange_id': '964c067c-46fb-4a4f-b30d-253029bf6d53',
 'presentation_request': {'name': 'Proof request',
                          'nonce': '236523038637111089936187',
                          'requested_attributes': {'data_dri': {'name': 'data_dri',
                                                                'non_revoked': {'from_epoch': 1500000000,
                                                                                'to_epoch': 1700758817},
                                                                'restrictions': [{'cred_def_id': 'WWZtMuY1V9hvFF1ezey2EM:3:CL:1175:Services'}]}},
                          'requested_predicates': {},
                          'version': '1.0'},
 'role': 'prover',
 'state': 'request_received',
 'thread_id': 'aab467d6-30db-442a-a23e-354f77100467',
 'trace': False,
 'updated_at': '2020-09-22 

In [360]:
url = 'http://localhost:8150/present-proof/records/' + \
       proof_request['presentation_exchange_id'] + \
       '/send-presentation'

presentation_schema = {
  "requested_attributes": {
    "data_dri": {
      "cred_id": credentials['results'][0]['referent'],
        "revealed": True
    },
  },
   "requested_predicates": {
  },
  "self_attested_attributes": {
  }
}
print(presentation_schema)
present = requests.post(url, json=presentation_schema)

print(present.text)

{'requested_attributes': {'data_dri': {'cred_id': '657ec1be-2d72-41ba-a1c2-725082d1c759', 'revealed': True}}, 'requested_predicates': {}, 'self_attested_attributes': {}}
{"initiator": "external", "presentation_exchange_id": "964c067c-46fb-4a4f-b30d-253029bf6d53", "created_at": "2020-09-22 11:58:22.954779Z", "trace": false, "state": "presentation_sent", "connection_id": "994ebd0f-14c2-428c-be53-e730bc3ca7a7", "presentation_request": {"version": "1.0", "name": "Proof request", "requested_attributes": {"data_dri": {"non_revoked": {"from_epoch": 1500000000, "to_epoch": 1700758817}, "restrictions": [{"cred_def_id": "WWZtMuY1V9hvFF1ezey2EM:3:CL:1175:Services"}], "name": "data_dri"}}, "requested_predicates": {}, "nonce": "236523038637111089936187"}, "thread_id": "aab467d6-30db-442a-a23e-354f77100467", "updated_at": "2020-09-22 11:58:42.143421Z", "presentation": {"proof": {"proofs": [{"primary_proof": {"eq_proof": {"revealed_attrs": {"data_dri": "46796680751272472783988362602137825638383598079

In [55]:
print(send.text)

{"hash_id": "ddd", "connection_id": "a361d994-aaea-4f5c-9bf4-9644a774de2b", "record_id": "1e61b917-946b-4a56-8810-5b48081f79e6", "exchange_id": "c5f71769-b806-49fe-8861-15f49ffad9e3", "state": "pending"}
